This notebook contains all the code necessary to generate the paper Ho, Perley et al. on ZTF18abvkwla (The Koala).

First, import all the packages that we will need.

In [104]:
from penquins import Kowalski
import numpy as np
from astropy.time import Time
import glob
import multiprocessing

# Query for analogs in ZTF

We will do these queries on Kowalski, so we log on there first.

In [7]:
def logon():
    """ Log onto Kowalski """
    username = 'ah'
    password = 'TetraodonInsists'
    s = Kowalski(
        protocol='https', host='kowalski.caltech.edu', port=443,
        verbose=False, username=username, password=password)
    return s

### Selection of field-nights

First I need to decide which field-nights to include in the search. We use the following criteria:

* At least one observation the night before (0.5 < dt < 1.5 days)
* At least one observation two nights before (2.5 < dt < 1.5 days)
* At least three observations in the next five nights (dt < 5.5 days)

In [33]:
def load():
    ut = []
    field = []
    filt = []
    ddir = "/Users/annaho/Dropbox/Projects/Research/Koala/data"
    for line in open("%s/allexp.tbl" %ddir, "r"):
        # not a header line
        if '1DC' in line:
            dat = line.split(' ')
            dat = [i for i in dat if i]
            ut.append(dat[0])
            field.append(dat[5])
            filt.append(dat[3])
    ut = np.array(ut)
    field = np.array(field)
    filt = np.array(filt)
    return ut,field,filt

In [39]:
def calc_n_active_nights():
    ut, field, filt = load()

    # Simplify the UT thing to just the date
    ut_day = np.array([i.split('T')[0] for i in ut])

    # Unique fields
    ufields = np.unique(field)

    # Number of active nights for that field
    nnights = np.zeros(len(ufields))

    # For each field, count the number of nights with 1 obs
    # AND an obs the night before AND an obs the night before that
    # AND 3 obs in the next 5 nights
    outputf = open("../data/fieldnights_1DC.txt", "w")
    for ii,uf in enumerate(ufields):
        print(uf)
        n_active_nights = 0
        field_day = Time(ut_day[field==uf], format='isot')
        unique_field_day = np.unique(field_day)
        for day in unique_field_day:
            nobs = sum(field_day==day) >= 1 # number of points in the night
            if nobs:
                dt = day-unique_field_day
                nobs_before = sum(np.logical_and(dt < 1.5, dt > 0.5)) >= 1
                nobs_2_before = sum(np.logical_and(dt < 2.5, dt > 1.5)) >= 1
                nobs_after = sum(np.logical_and(dt > -5.5, dt < -0.5)) >= 3
                if np.logical_and(nobs_before, nobs_after):
                    n_active_nights += 1
                    print(uf, str(day).split('T')[0])
                    outputf.write("%s,%s" %(uf, str(day).split('T')[0]))
                    outputf.write('\n')
        print(n_active_nights)
        nnights[ii] = n_active_nights
    outputf.close()

    # Save as array
    np.savetxt("../data/n_active_nights.txt",
            np.array([ufields,nnights]).T, fmt='%s',
            delimiter=',')

In [42]:
calc_n_active_nights()

245
245 2018-08-13
245 2018-08-14
245 2018-08-15
245 2018-08-18
245 2018-08-19
245 2018-08-30
245 2018-08-31
245 2018-09-01
245 2018-09-02
245 2018-09-03
245 2018-09-04
245 2018-09-05
245 2018-09-06
245 2018-09-07
245 2018-09-08
245 2018-09-09
245 2018-09-10
245 2018-09-11
245 2018-09-12
245 2018-09-13
245 2018-09-14
245 2018-09-15
245 2018-09-16
245 2018-09-17
245 2018-09-18
245 2018-09-19
245 2018-09-22
245 2018-09-23
245 2018-09-26
29
246
246 2018-08-14
246 2018-08-18
246 2018-08-19
246 2018-08-20
246 2018-08-24
246 2018-08-25
246 2018-08-26
246 2018-08-27
246 2018-08-28
246 2018-08-29
246 2018-08-30
246 2018-08-31
246 2018-09-01
246 2018-09-02
246 2018-09-03
246 2018-09-04
246 2018-09-05
246 2018-09-06
246 2018-09-09
246 2018-09-10
246 2018-09-11
246 2018-09-12
246 2018-09-13
246 2018-09-16
246 2018-09-17
246 2018-09-22
246 2018-09-23
246 2018-09-24
246 2018-09-25
246 2018-09-26
30
247
247 2018-08-25
247 2018-08-26
247 2018-08-27
247 2018-08-28
247 2018-08-29
247 2018-08-30
247 201

302 2018-09-22
302 2018-09-23
302 2018-09-26
302 2019-01-04
302 2019-01-05
302 2019-01-08
302 2019-01-23
302 2019-01-24
27
303
303 2019-01-04
303 2019-01-05
303 2019-01-08
303 2019-01-25
4
304
304 2018-11-01
304 2018-11-02
304 2018-11-03
304 2018-11-04
304 2018-11-05
304 2018-11-06
304 2018-11-07
304 2018-11-08
304 2018-11-09
304 2018-11-10
304 2019-01-04
304 2019-01-05
304 2019-01-08
13
305
305 2018-11-01
305 2018-11-02
305 2018-11-03
305 2018-11-04
305 2018-11-05
305 2018-11-06
305 2018-11-07
305 2018-11-08
305 2018-11-09
305 2018-11-10
305 2019-01-05
305 2019-01-08
305 2019-01-23
305 2019-01-24
305 2019-01-25
15
306
306 2018-11-01
306 2018-11-02
306 2018-11-03
306 2018-11-04
306 2018-11-05
306 2018-11-08
306 2018-11-09
306 2018-11-10
306 2018-11-13
306 2018-11-14
306 2018-11-15
306 2018-11-16
12
307
307 2018-11-01
307 2018-11-04
307 2018-11-05
307 2018-11-10
307 2018-11-13
307 2018-11-14
307 2018-11-15
307 2018-11-16
307 2019-03-15
9
313
0
314
314 2018-12-17
314 2019-01-23
314 2019-

363 2018-12-15
363 2018-12-16
363 2018-12-17
15
364
364 2018-11-01
364 2018-11-04
364 2018-11-05
364 2018-11-08
364 2018-11-09
364 2018-11-10
364 2018-11-15
364 2018-11-16
364 2018-12-14
364 2018-12-15
10
365
365 2018-12-14
365 2018-12-15
2
366
366 2018-12-13
366 2018-12-14
366 2018-12-15
3
367
367 2018-12-13
367 2018-12-14
367 2018-12-15
3
368
368 2018-12-14
368 2018-12-15
2
374
0
388
388 2018-11-02
388 2018-11-03
388 2018-11-04
388 2018-11-05
388 2019-07-29
388 2019-07-30
388 2019-07-31
388 2019-08-03
388 2019-08-17
9
389
389 2018-11-03
389 2018-11-04
389 2018-11-05
389 2018-11-06
389 2018-11-07
389 2018-11-08
389 2019-07-27
389 2019-07-28
389 2019-07-29
389 2019-08-01
389 2019-08-02
389 2019-08-03
389 2019-08-04
13
392
392 2018-11-02
392 2018-11-03
392 2018-11-06
392 2018-11-07
392 2018-11-08
392 2018-11-09
392 2018-11-10
392 2018-11-11
392 2019-07-27
392 2019-07-28
392 2019-07-29
392 2019-07-30
392 2019-07-31
392 2019-08-01
392 2019-08-02
392 2019-08-03
392 2019-08-04
392 2019-08-0

8
418
0
425
425 2018-12-17
1
426
426 2019-01-20
1
427
427 2019-01-20
427 2019-04-09
427 2019-04-10
427 2019-04-11
427 2019-07-15
427 2019-07-16
427 2019-07-17
427 2019-07-18
427 2019-07-19
427 2019-07-20
427 2019-07-21
11
439
439 2018-11-04
439 2018-11-05
439 2018-11-06
439 2019-07-27
439 2019-07-28
439 2019-07-29
439 2019-07-30
439 2019-07-31
439 2019-08-03
439 2019-08-06
439 2019-08-17
11
443
443 2018-11-02
443 2018-11-03
443 2018-11-04
443 2018-11-05
443 2018-11-06
443 2018-11-07
443 2018-11-08
443 2018-11-09
443 2018-11-10
443 2018-11-11
443 2019-07-14
443 2019-07-27
443 2019-07-28
443 2019-07-29
443 2019-07-30
443 2019-07-31
443 2019-08-01
443 2019-08-02
443 2019-08-03
443 2019-08-04
443 2019-08-05
443 2019-08-06
443 2019-08-07
443 2019-08-08
24
448
448 2018-08-16
448 2018-08-17
448 2018-08-18
448 2018-08-19
448 2018-08-20
448 2018-08-21
448 2018-08-22
448 2018-08-23
448 2018-08-24
448 2018-09-01
448 2018-09-02
448 2018-09-03
448 2018-09-04
448 2018-09-05
448 2018-09-06
448 2018-0

498 2019-08-04
498 2019-08-05
498 2019-08-06
498 2019-08-09
498 2019-08-10
498 2019-08-13
55
499
499 2018-08-16
499 2018-08-17
499 2018-08-18
499 2018-08-19
499 2018-08-20
499 2018-08-21
499 2018-08-22
499 2018-08-23
499 2018-08-24
499 2018-08-25
499 2018-08-26
499 2018-08-27
499 2018-08-31
499 2018-09-01
499 2018-09-02
499 2018-09-03
499 2018-09-04
499 2018-09-05
499 2018-09-06
499 2018-09-07
499 2018-09-08
499 2018-09-09
499 2018-09-10
499 2018-09-11
499 2018-09-12
499 2018-09-13
499 2018-09-14
499 2018-09-15
499 2018-09-16
499 2018-09-17
499 2018-09-18
499 2018-09-19
499 2018-09-22
499 2018-09-23
499 2018-09-24
499 2018-11-01
499 2018-11-02
499 2018-11-03
499 2018-11-04
499 2018-11-05
499 2018-11-06
499 2018-11-07
499 2018-11-08
499 2018-11-09
499 2018-11-10
499 2018-11-13
499 2019-01-05
499 2019-01-08
499 2019-07-14
499 2019-07-15
499 2019-07-16
499 2019-07-17
499 2019-07-18
499 2019-07-19
499 2019-07-27
499 2019-07-30
499 2019-07-31
499 2019-08-01
499 2019-08-02
499 2019-08-03
499

...

718 2018-04-22
718 2018-04-23
718 2018-04-24
718 2018-04-25
718 2018-04-26
718 2018-05-05
718 2018-05-06
718 2018-05-07
718 2018-05-08
718 2018-05-09
16
719
719 2018-04-19
719 2018-04-22
719 2018-04-23
719 2018-04-24
719 2018-04-25
719 2018-04-26
719 2018-05-05
719 2018-05-06
719 2018-05-07
719 2018-05-08
719 2018-05-18
11
720
720 2018-04-19
720 2018-04-22
720 2018-04-23
720 2018-04-24
720 2018-04-25
720 2018-04-26
720 2018-05-05
720 2018-05-06
720 2018-05-07
720 2018-05-08
720 2018-05-18
11
721
721 2018-04-09
721 2018-04-19
721 2018-04-22
721 2018-04-23
721 2018-04-24
721 2018-04-25
721 2018-04-26
721 2018-05-05
721 2018-05-06
721 2018-05-07
721 2018-05-08
721 2018-05-17
12
722
722 2018-04-09
722 2018-04-19
722 2018-04-22
722 2018-04-23
722 2018-04-24
722 2018-04-25
722 2018-04-26
722 2018-05-05
722 2018-05-06
722 2018-05-07
722 2018-05-08
722 2018-05-17
12
727
727 2018-08-11
1
728
728 2018-08-11
1
731
0
732
732 2019-09-07
1
733
733 2019-09-07
733 2019-09-08
2
734
734 2018-08-13
734 2

752 2018-04-16
752 2018-04-26
752 2018-04-27
752 2018-05-05
752 2018-05-06
752 2018-05-07
752 2018-05-08
752 2018-05-09
752 2018-05-20
752 2018-05-29
752 2018-06-09
752 2018-06-10
752 2018-06-11
752 2018-11-01
752 2018-11-02
752 2018-11-03
752 2018-11-04
752 2018-11-05
752 2018-11-06
752 2018-11-07
752 2018-11-08
752 2018-11-09
752 2018-11-10
752 2018-11-13
752 2018-11-14
752 2018-11-15
752 2018-11-16
752 2018-12-10
752 2018-12-13
752 2018-12-14
752 2018-12-15
752 2018-12-16
752 2018-12-17
752 2019-01-20
752 2019-01-23
752 2019-01-24
752 2019-01-25
752 2019-01-26
752 2019-01-27
752 2019-01-28
44
753
753 2018-04-08
753 2018-04-09
753 2018-04-19
753 2018-04-23
753 2018-04-24
753 2018-04-25
753 2018-04-26
753 2018-04-27
753 2018-05-05
753 2018-05-06
753 2018-05-07
753 2018-05-08
753 2018-05-09
753 2018-05-17
753 2018-05-20
753 2018-05-29
753 2018-05-30
753 2018-05-31
753 2018-06-03
753 2018-06-08
753 2018-06-09
753 2018-06-12
753 2018-11-01
753 2018-11-02
753 2018-11-03
753 2018-11-04
753

786 2019-01-28
786 2019-02-25
786 2019-03-16
786 2019-03-17
786 2019-04-08
786 2019-04-09
786 2019-04-12
786 2019-04-13
786 2019-04-14
786 2019-04-15
786 2019-04-16
786 2019-04-19
786 2019-04-24
786 2019-04-25
786 2019-04-26
786 2019-04-27
786 2019-04-28
53
787
787 2018-04-07
787 2018-04-08
787 2018-04-09
787 2018-04-10
787 2018-04-11
787 2018-04-22
787 2018-04-23
787 2018-04-24
787 2018-04-25
787 2018-04-26
787 2018-04-27
787 2018-05-05
787 2018-05-06
787 2018-05-07
787 2018-05-08
787 2018-05-09
787 2018-05-20
787 2018-05-29
787 2018-05-30
787 2018-05-31
787 2018-06-03
787 2018-06-08
787 2018-06-09
787 2018-06-10
787 2018-06-11
787 2018-06-12
787 2018-11-01
787 2018-11-02
787 2018-11-03
787 2018-11-04
787 2018-11-05
787 2018-11-06
787 2018-11-07
787 2018-11-08
787 2018-11-09
787 2018-11-10
787 2018-11-15
787 2018-11-16
787 2018-12-10
787 2018-12-13
787 2018-12-14
787 2018-12-15
787 2018-12-16
787 2018-12-17
787 2019-01-20
787 2019-01-23
787 2019-01-24
787 2019-01-25
787 2019-01-26
787

814 2018-11-03
814 2018-11-04
814 2018-11-05
814 2018-11-06
814 2018-11-07
814 2018-11-08
814 2018-11-09
814 2018-11-10
814 2018-11-13
814 2018-11-14
814 2018-11-15
814 2018-11-16
814 2018-12-10
814 2018-12-13
814 2018-12-14
814 2018-12-15
814 2018-12-16
814 2018-12-17
814 2018-12-20
814 2019-02-25
814 2019-03-15
814 2019-03-16
814 2019-03-17
814 2019-04-09
814 2019-04-12
814 2019-04-13
814 2019-04-14
814 2019-04-15
814 2019-04-16
59
815
815 2018-11-01
815 2018-11-02
815 2018-11-03
815 2018-11-04
815 2018-11-05
815 2018-11-06
815 2018-11-07
815 2018-11-08
815 2018-11-09
815 2018-11-10
815 2018-11-13
815 2018-11-14
815 2018-11-15
815 2018-11-16
815 2018-12-10
815 2018-12-13
815 2018-12-14
815 2018-12-15
815 2018-12-16
815 2018-12-17
815 2019-01-04
815 2019-01-08
815 2019-01-21
815 2019-01-22
815 2019-01-23
815 2019-01-24
815 2019-01-25
815 2019-01-26
815 2019-01-27
815 2019-01-28
815 2019-02-25
815 2019-03-15
815 2019-03-16
815 2019-03-17
815 2019-04-09
815 2019-04-12
815 2019-04-13
815

838 2018-12-15
838 2018-12-16
838 2018-12-17
838 2018-12-20
838 2019-03-15
838 2019-03-16
838 2019-03-17
838 2019-04-09
838 2019-04-12
838 2019-04-13
838 2019-04-14
67
839
839 2018-08-26
839 2018-08-27
839 2018-08-28
839 2018-08-29
839 2018-08-30
839 2018-08-31
839 2018-09-01
839 2018-09-02
839 2018-09-03
839 2018-09-04
839 2018-09-05
839 2018-09-06
839 2018-09-07
839 2018-09-08
839 2018-09-09
839 2018-09-10
839 2018-09-11
839 2018-09-12
839 2018-09-13
839 2018-09-14
839 2018-09-15
839 2018-09-16
839 2018-09-17
839 2018-09-18
839 2018-09-19
839 2018-09-22
839 2018-09-23
839 2018-09-24
839 2018-09-25
839 2018-09-26
839 2018-11-01
839 2018-11-02
839 2018-11-03
839 2018-11-04
839 2018-11-05
839 2018-11-06
839 2018-11-07
839 2018-11-08
839 2018-11-09
839 2018-11-10
839 2018-11-13
839 2018-11-14
839 2018-11-15
839 2018-11-16
839 2018-12-13
839 2018-12-14
839 2018-12-15
839 2018-12-16
839 2018-12-17
839 2018-12-20
839 2019-03-15
839 2019-03-16
839 2019-03-17
839 2019-04-09
839 2019-04-12
839

In [47]:
dat = np.loadtxt("../data/n_active_nights.txt", delimiter=',')
sum(dat[:,1])

8064.0

The total is 8064 field-days, and we save the field-nights to a file in the Koala/data directory. We will use this file to run our queries.

For each field-night, we will do a very loose query for all sources that are real (drb > 0.99) and not stars. We will save that list to a file, and use this file to perform a more detailed query later.

In [73]:
def get_query(start,end,field):
    q = {"query_type": "find",
         "query": {
             "catalog": "ZTF_alerts",
             "filter": {
                     'candidate.jd': {'$gt': start, '$lt': end},
                     'candidate.field': {'$eq': field},
                     'classifications.braai': {'$gt': 0.99},
                     'candidate.magpsf': {'$lt': 20},
                     'candidate.isdiffpos': {'$in': ['1', 't']},
             },
             "projection": {
                     "objectId": 1,
                     "candidate.ra": 1,
                     "candidate.dec": 1,
                     "candidate.distpsnr1": 1,
                     "candidate.sgscore1": 1,
             },
         "kwargs": {
             "hint": "jd_field_rb_drb_braai_ndethhist_magpsf_isdiffpos",
         }
         }
         }
    return q

In [96]:
def search(s, field, start, end):
    q = get_query(start,end,field)
    r = s.query(query=q)
    out = r['result_data']['query_result']
    
    # Get rid of stars 
    distnr = np.array([d['candidate']['distpsnr1'] for d in out])
    sgscore = np.array([d['candidate']['sgscore1'] for d in out])
    star = np.logical_and(distnr<1.0, sgscore>0.76)
    oids = np.array([d['objectId'] for d in out])[~star]
    
    # Return unique candidates
    oids_unique, inds = np.unique(oids, return_index=True)
    ras = np.array([d['candidate']['ra'] for d in out])[~star][inds]
    decs = np.array([d['candidate']['dec'] for d in out])[~star][inds]

    return oids_unique

In [56]:
class Day:
    """ Get a date as a datestr, as a time object, and get to the next day """
    def __init__(self, date):
        self.timeobj = Time(date, format='isot')
        self.datestr = self.timeobj.iso[0:10].replace('-','')

    def next_day(self):
        timeobj = Time(self.timeobj.jd + 1, format='jd')
        day2 = Day(timeobj.iso[0:10])
        return day2

In [97]:
dat = np.loadtxt("../data/fieldnights_1DC.txt", dtype=str, delimiter=',')
fld = dat[:,0].astype(int)
date = [Day(val) for val in dat[:,1]]

In [ ]:
for ii,fld_num in enumerate(fld):
    print(100*ii/len(fld))
    outf = "output/%s_%s.txt" %(date[ii].datestr,fld_num)

    if len(glob.glob(outf)) == 0:
        #print("Running for %s,%s" %(date[ii].datestr,fld_num))
        start = date[ii].timeobj.jd
        end = start+1
        s = logon()
        oids = search(s,int(fld_num),start,end)

        print("%s passed" %len(oids))
        #print(oids)

        np.savetxt(outf, oids, fmt='%s')
    else:
        print("already run")

0.0
already run
0.01240079365079365
already run
0.0248015873015873
already run
0.03720238095238095
already run
0.0496031746031746
already run
0.062003968253968256
already run
0.0744047619047619
already run
0.08680555555555555
already run
0.0992063492063492
already run
0.11160714285714286
already run
0.12400793650793651
already run
0.13640873015873015
already run
0.1488095238095238
already run
0.16121031746031747
already run
0.1736111111111111
already run
0.18601190476190477
already run
0.1984126984126984
already run
0.21081349206349206
already run
0.22321428571428573
already run
0.23561507936507936
already run
0.24801587301587302
already run
0.2604166666666667
already run
0.2728174603174603
already run
0.28521825396825395
already run
0.2976190476190476
already run
0.3100198412698413
already run
0.32242063492063494
already run
0.33482142857142855
already run
0.3472222222222222
already run
0.35962301587301587
already run
0.37202380952380953
already run
0.3844246031746032
already run
0.39

97 passed
3.5342261904761907
233 passed
3.546626984126984
266 passed
3.5590277777777777
276 passed
3.5714285714285716
282 passed
3.583829365079365
314 passed
3.5962301587301586


Let's calculate the number of sources that passed this initial broad query.

In [ ]:
files = glob.glob("output/*.txt")
nsrc = []
for f in files:
    nsrc += len(np.loadtxt(f))

The total number of sources that passed this initial broad query is XX.

For each of these sources, let's perform a more detailed query on the light curve.

In [ ]:
files = glob.glob("output/*.txt")
for f in files:
    string = f.
    nsrc += len(np.loadtxt(f))